In [1]:
import numpy as np
import cv2
from os import listdir
from matplotlib import pyplot as plt
import PIL.Image as pil
import pandas as pd

In [2]:
# Import custom methods
import general_methods
import semantic_segmentation

c:\Users\Kim\anaconda3\envs\pipeline1\lib\site-packages\gluoncv\__init__.py:40: UserWarning: Both `mxnet==1.7.0` and `torch==1.6.0+cpu` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


## Image processing & results gathering

In [50]:
# Get pre-trained models
seg_model = semantic_segmentation.get_pretrained_model()

In [51]:
# Method to perform segmentation and return black image with only walls in grey

def wall_segmentation(filename):
    input_pil = general_methods.import_and_resize(filename)
    input_img = general_methods.import_mx_image(filename)
    input_cv2 = general_methods.import_cv2_image(filename)
    height = input_cv2.shape[0]
    width = input_cv2.shape[1]
    size = (width, height)

    # Segmentation
    mmask = semantic_segmentation.get_segementation(input_img, seg_model)
    labels = general_methods.get_labels_string(mmask, mmask.shape[0])
    walls = general_methods.find_colour_indices(labels, "0.47058824,0.47058824,0.47058824,1.0")
    black = np.zeros((height, width, 3))
    segmented_input = semantic_segmentation.remove_inds(black, [walls], [120, 120, 120])

    return segmented_input

In [ ]:
# Change seg images so only walls are grey

folder_dir = "images/seg-images/"
 
for image in listdir(folder_dir):
    filename = "images/seg-images/" + image
    print(image)
    input_cv2 = general_methods.import_cv2_image(filename)
    img = input_cv2.copy()
    height = input_cv2.shape[0]
    width = input_cv2.shape[1]
    for i in range(height):
        for j in range(width):
            if img[i][j][0] == 0:
                img[i][j] = [0, 0, 0]
            if img[i][j][0] != 0:
                img[i][j] = [120, 120, 120]
    
    cv2.imwrite("images/seg-images-after/" + image, img) 

In [ ]:
for image in listdir("data-collection/groundtruths/segmentation/"):
    filename = "data-collection/groundtruths/segmentation/" + image
    image_name = image[:-13]
    print(image)
    input_cv2 = general_methods.import_cv2_image(filename)
    height = input_cv2.shape[0]
    width = input_cv2.shape[1]

    input_pil = pil.open("images/outputs/segmentation/" + image_name + ".jpg")
    input_pil = input_pil.resize((width, height))
    input_pil.save("images/outputs/segmentation-resized/" + image_name + ".jpg")

    input_pil = pil.open("images/seg-images/" + image_name + ".png")
    input_pil = input_pil.resize((width, height))
    input_pil.save("images/seg-images-resized/" + image_name + ".png")

ADE_val_00000079_wall_seg.png


In [ ]:
for image in listdir("images/outputs/segmentation/"):
    filename = "images/outputs/segmentation/" + image
    image_name = image[:-4]
    print(image)
    input_cv2 = general_methods.import_cv2_image(filename)
    height = input_cv2.shape[0]
    width = input_cv2.shape[1]

    try:
        img1 = cv2.imread("images/seg-images/" + image_name + ".png")
        img1 = cv2.resize(img1,(width, height),fx=0, fy=0, interpolation = cv2.INTER_NEAREST)
        cv2.imwrite("images/seg-images-resized/" + image_name + ".png", img1)
    except:
        print(image_name + " doesn't exist")

    img2 = cv2.imread("data-collection/groundtruths/segmentation/" + image_name + "_wall_seg.png")
    img2 = cv2.resize(img2,(width, height),fx=0, fy=0, interpolation = cv2.INTER_NEAREST)
    cv2.imwrite("data-collection/groundtruths/segmentation-resized/" + image_name + "_wall_seg.png", img2)

## Intersection over Union

In [54]:
mine = general_methods.import_cv2_image("images/outputs/segmentation/ADE_val_00000079.jpg")
wiz = general_methods.import_cv2_image("images/seg-images-after/ADE_val_00000079.png")
gt = general_methods.import_cv2_image("data-collection/groundtruths/segmentation-resized/ADE_val_00000079_wall_seg.png")

In [3]:
def get_iuo(image_1, image_2):
    # https://towardsdatascience.com/intersection-over-union-iou-calculation-for-evaluating-an-image-segmentation-model-8b22e2e84686
    intersection = np.logical_and(image_1, image_2)
    union = np.logical_or(image_1, image_2)
    iou = np.sum(intersection) / np.sum(union)
    return iou

In [11]:
df = pd.DataFrame(data=[], columns=["Image_name", "Pipeline_GT_IOU", "Wizart_GT_IOU"])

In [12]:
for image in listdir("images/outputs/segmentation/"):
    filename = "images/outputs/segmentation/" + image
    image_name = image[:-4]
    pl = general_methods.import_cv2_image(filename)
    gt = general_methods.import_cv2_image("data-collection/groundtruths/segmentation-resized/" + image_name + "_wall_seg.png")
    pl_iou = round(get_iuo(gt, pl) * 100, 1)

    try:
        wiz = general_methods.import_cv2_image("images/seg-images-resized/" + image_name + ".png")
        wiz_iou = round(get_iuo(gt, wiz) * 100, 1)
    except:
        wiz_iou = float("nan")

    new_data = [image_name, pl_iou, wiz_iou]
    df.loc[len(df)] = new_data
    
    

In [13]:
df

,Image_name,Pipeline_GT_IOU,Wizart_GT_IOU
0,ADE_val_00000079,78.9,58.5
1,ADE_val_00000085,88.0,53.6
2,ADE_val_00000091,76.0,59.8
3,ADE_val_00000092,67.6,NaN
4,ADE_val_00000093,93.7,78.3
5,ADE_val_00000101,33.0,42.4
6,ADE_val_00000105,64.8,NaN
7,ADE_val_00000119,90.7,NaN
8,ADE_val_00000141,93.9,83.5
9,ADE_val_00000151,88.2,74.5


In [ ]:
# How many was mine better in?

# Average wall coverage percentage for both

# Standard deviation for both

# Test both for Normality
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.normaltest.html
# If p value greater than 0.05 then it's normally distributed

# Wilcoxon test or Paired t test (to test significance)

In [14]:
df.to_csv("data-collection/outputs/segmentation-iou-2.csv", index=False)